### Setup

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
CRYPTO_TARGET = 'MBEDTLS'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-rsa
#make clean PLATFORM=$1 CRYPTO_TARGET=$2
make PLATFORM=$1 CRYPTO_TARGET=$2 

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CW308_STM32F3.hex'

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
import numpy as np
import time 
import matplotlib.pylab as plt
import matplotlib

In [ ]:
e = 65537

In [ ]:

n = 89525655797379415138998373505682772229

p = 14091828191315287507 
q = 6353019252147394247
dp = 5927267588107438781
dq = 2516023234043145363
qinv = 907488296688955759

In [ ]:
#8Bytes -> key_len = 16
n = 89525655797379415138998373505682772229

In [ ]:
#key lenght of n; p, q have lenght key_len//2
key_len = 16

### find numbers

In [ ]:
# Parameters
p = 14091828191315287507 
q = 6353019252147394247
    9223372036854775808
     288230376151711744
e = 65537
n = 89525655797379415138998373505682772229
max_msg = 10000  # Set a range for msg (you can adjust this range)

In [ ]:
def find_valid_msg_d1(p, q, e, n, max_msg):
    valid_msgs = []
    for msg in range(1, max_msg + 1):  # Iterate through possible msg values
        x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) < 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs



# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

In [ ]:
def find_valid_msgs_d0(p, q, e, n, max_msg):
    valid_msgs = []
    for msg in range(1, max_msg + 1):  # Iterate through possible msg values
        x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) > 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs

# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

### capture trace setup

In [ ]:
scope.clock.adc_src = "clkgen_x1"
scope.adc.samples = 650 #print: 631

#scope.adc.timeout = 50

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    scope.arm()
    target.simpleserial_write('t', text)

    # get debug output, nötig?
    #rec_str = target.read(1000, timeout=2000)
    #received_str = rec_str.replace("\\r\\n", "\n")
    #print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    time.sleep(2)
    output = target.read(timeout=20)
    print('first: ', output.replace("\\r\\n\\z00", "\n"))
    
    target.write("1\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('next 48 bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))

    target.write("2\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('last bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))


    #newout = output.replace("r", "").replace("\nz00","").replace("\n","")
    #print('output: ', newout)

    if SCOPETYPE == "OPENADC":
        print('#samples: ', scope.adc.trig_count)
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    target.flush()
    scope.arm()
    target.simpleserial_write('t', text)

    rec_str = target.read(1000, timeout=200)
    received_str = rec_str.replace("\\r\\n", "\n")
    print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

### test numbers

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 54
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace1 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace1, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 16
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt
#p= 14091828191315287507

msg = 152261522452152221552
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace3 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace3, 'r')


In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 62549 
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace4 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace4, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt
#p= 14091828191315287507
msg = 215675565258552152214
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace5 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace5, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt
#p= 14091828191315287507
msg = 215675565258586152214
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace6 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace6, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt
#p= 14091828191315287507
msg = 9223372036854775808
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace7 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace7, 'r')

In [ ]:
print(np.sum(np.abs(trace1 - trace2))) #same
print(np.sum(np.abs(trace7 - trace3))) #same
print(np.sum(np.abs(trace7 - trace4))) #same
print(np.sum(np.abs(trace2 - trace3))) #same
print(np.sum(np.abs(trace2 - trace4))) #same
print(np.sum(np.abs(trace3 - trace4))) #same 

print(np.sum(np.abs(trace5 - trace6))) #same 

print(np.sum(np.abs(trace3 - trace5))) #
print(np.sum(np.abs(trace3 - trace6))) #

In [ ]:
plt.figure()
plt.plot(trace1, 'r')
plt.plot(trace2, 'b')
plt.plot(trace5, 'y')
plt.plot(trace7, 'g')

In [ ]:
# keylen = 4
# #samples für MBEDTLS_MPI_CHK( mbedtls_mpi_sub_mpi( &T, &T1, &T2 ) ) = 600
# diff(same): 2 
# diff(diff): 30

### Key Reconstuction

* find plaintexts x_1 and x_2 mit diff(x_1) = 0 und diff(x_2) = 1.
* mittels binäre suche: finde x mit diff(x-1) = 0 und diff(x) = 1 -> x ist Vielfaches von p 
* ggt(x, n) = p
* x^e mod n

### Algorithmus

Reconstruction of p when its bitlength is half bitlength of n
INPUT: modulus n with the bitlength t, public exponent e.
OUTPUT: prime p such that p divides n.

t = bitlenght von n 
p,q haben bitlänge t/2

x = 2 ** (t/2)
m = x/2 

In [ ]:
#  Byte Primzahlen 

t = key_len*8

# x= 2 ** (t/2) 
# m = x/2 
# l = 0 

#upper
x = 2**(t//2)
m = x//2
#lower
l = 0

print(x, m, l)

In [ ]:
plt.figure()

ref_trace_msg_0 = 152261522452152221552 
c_0 = (ref_trace_msg_0 ** e) % n 
msg_0 = c_0.to_bytes(key_len,'big')
trace_0 = capture_RSA_trace(scope, target, msg_0)
plt.plot(trace_0, 'r')

ref_trace_msg_1 = 215675565258586152214 
c_1 = (ref_trace_msg_1 ** e) % n 
msg_1 = c_1.to_bytes(key_len,'big')
trace_1 = capture_RSA_trace(scope, target, msg_1)
plt.plot(trace_1, 'b')


print(np.sum(np.abs(trace_1 - trace_0)))

In [ ]:
plt.figure()

while m != l:
    print("new Value: ")
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(key_len,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    print(m, hex(enc))

    diff0 = np.sum(np.abs(trace - trace_0))
    diff1 = np.sum(np.abs(trace - trace_1))
    
    print("diff0: ", diff0)
    print("diff1: ", diff1)
    
    if diff1 < 15:  # ähnliche -> diff(m) = 1
        plt.plot(trace, 'g')
        x = m
    elif diff0 > 15: # nicht diff0
        plt.plot(trace, 'r')
        x = m
    elif diff0 < 15: # ähnlich diff0
        plt.plot(trace, 'b')
        l = m
        
    m = (l + x)//2
    
    

print (x)
print (p)

#bei 0x...3b14... kommt 3b15...falsch heraus
# -> or diff0 > 15 -> fix
# diff0 bessere ref trace ?!

In [ ]:
plt.figure()

trace_0_w = trace_0[60:]
trace_1_w = trace_1[60:]

trace_1_s = trace_1_w[:-30]

while m != l:
    print("new Value: ")
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(key_len,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    print(m, hex(enc))


    diff0 = np.sum(np.abs(trace - trace_0))
    diff1 = np.sum(np.abs(trace - trace_1))
    
    print("diff0: ", diff0)
    print("diff1: ", diff1)

    trace = trace[60:]
    
    #vgl zu diff1
    min_sad_1 = 50
    for offset in range(31):
        diff = trace_1_s - trace[offset:offset + len(trace_1_s)]
        sad = np.sum(np.abs(diff))
        if sad < min_sad_1:
            min_sad_1 = sad
    print("min sad 1: ", min_sad_1)
    
    if diff1 < 15:  # ähnliche -> diff(m) = 1
        plt.plot(trace, 'g')
        x = m
    elif diff0 > 15: # nicht diff0
        plt.plot(trace, 'r')
        x = m
    elif diff0 < 15: # ähnlich diff0
        plt.plot(trace, 'b')
        l = m
        
    m = (l + x)//2
    
    

print (x)
print (p)


p = 14091828191315287507
q = 6353019252147394247
N = 89525655797379415138998373505682772229

q = N // p
euler = (p-1)*(q-1)

-> d durch erweiterter euklidischer algo 

In [ ]:
scope.dis()
target.dis()